**SemEval 2019**

The Dataset includes two different sets of articles for partisan news detection.
The first are 750,000 articles annotated on article level by allsides.
The second a 645 articles collection of human annotated articles (only those with inter-annotator agreement).
Since in the BigNews Corpus a lot of allsides articles were already included the focus here was on the second dataset, resulting in 9783 annotated sentences.

```
LABEL:
Neutral - 0
Hyperpartisan - 1
```

In [1]:
import os
import sys
import pandas as pd
from prep_collection import PrepCollection as prep
# import numpy as np
import json
from bs4 import BeautifulSoup as bs

In [2]:
wdr_path = os.path.dirname(os.path.dirname(os.getcwd()))
ds_raw_path = os.path.join(wdr_path + "/Datasets/Fake News/SemEval 2019 Task 4")
articles = [os.path.join(ds_raw_path, 'articles-training-byarticle-20181122.xml'), os.path.join(ds_raw_path, 'articles-test-byarticle-20181207.xml')]
ground_truth = [os.path.join(ds_raw_path, 'ground-truth-training-byarticle-20181122.xml'), os.path.join(ds_raw_path, 'ground-truth-test-byarticle-20181207.xml')]

In [3]:
def preprocess_semeval(articles, ground_truth, wdr_path):
    df_final = pd.DataFrame(columns=['id', 'text', 'label'])
    for z in range(len(articles)):
        with open(articles[z], 'r', errors='replace') as f:
            file = f.read()
        bs_file = bs(file, 'xml')
        ds = []
        for article in bs_file.find_all('article'):
            article_id = article.get('id')
            sentences = [prep.prepare_text(ele.get_text()) for ele in article.find_all('p')]
            for k, sentence in enumerate(sentences):
                ds.append([str(article_id)+'-'+str(k), article_id, sentence, len(sentence.split())])
        df_articles = pd.DataFrame(ds, columns= ['id', 'article_id', 'text', 'wordcount'])
        with open(ground_truth[z], 'r', errors='replace') as g:
            file2 = g.read()
        bs_file_label = bs(file2, 'xml')
        ds_label = []
        for labels in bs_file_label.find_all('article'):
            article_id = labels.get('id')
            hyperpartisan = labels.get('hyperpartisan')
            if hyperpartisan == "true":
                label = 1
            elif hyperpartisan == "false":
                label = 0
            ds_label.append([article_id, label])
        df_labels = pd.DataFrame(ds_label, columns= ['article_id', 'label'])
        df = pd.merge(df_articles, df_labels, on= 'article_id', how= 'inner')
        df.drop('wordcount', axis=1, inplace=True)
        df.drop('article_id', axis=1, inplace=True)
        df_final = pd.concat([df_final, df])
    df_final.to_csv(os.path.join(wdr_path + "/Preprocessed_Datasets/029-SemEval2019.csv"))
    return df_final

In [4]:
df = preprocess_semeval(articles, ground_truth, wdr_path)